In [2]:
import pandas as pd

project_path = '/home/wjunneng/Ubuntu/2019-CCF-Passenger-Car-Segment-Sales-Forecast'

# train_sales_data
train_sales_data_path = project_path + '/data/original/train_sales_data.csv'
# train_search_data
train_search_data_path = project_path + '/data/original/train_search_data.csv'
# train_user_reply_data
train_user_reply_data = project_path + '/data/original/train_user_reply_data.csv'

# evaluation_public
evaluation_public = project_path + '/data/original/evaluation_public.csv'
# submit_example.csv
submit_example = project_path + '/data/original/submit_example.csv'

train_sales_data = pd.read_csv(train_sales_data_path)
train_search_data = pd.read_csv(train_search_data_path)
train_user_reply_data = pd.read_csv(train_user_reply_data)
evaluation_public = pd.read_csv(evaluation_public)




In [3]:
import pandas as pd

# 合并了sales + search
train_sales_search = pd.merge(train_sales_data, train_search_data, on=['province','adcode','model','regYear', 'regMonth'])
# 合并了sales + search + user_reply
train_sales_search_user_reply = pd.merge(train_sales_search, train_user_reply_data, on=['model', 'regYear', 'regMonth'])

print(train_sales_search.columns)
print(evaluation_public.columns)

del evaluation_public['forecastVolum']
del evaluation_public['province']
del train_sales_search['province']
del train_sales_search['bodyType']
del train_sales_search['popularity']

X_test_id = evaluation_public['id']
del evaluation_public['id']

X = pd.concat([train_sales_search, evaluation_public], axis=0)
print(X.shape)
print(X.head())




Index(['province', 'adcode', 'model', 'bodyType', 'regYear', 'regMonth',
       'salesVolume', 'popularity'],
      dtype='object')
Index(['id', 'province', 'adcode', 'model', 'regYear', 'regMonth',
       'forecastVolum'],
      dtype='object')
(36960, 5)
   adcode             model  regMonth  regYear  salesVolume
0  310000  3c974920a76ac9c1         1     2016        292.0
1  530000  3c974920a76ac9c1         1     2016        466.0
2  150000  3c974920a76ac9c1         1     2016        257.0
3  110000  3c974920a76ac9c1         1     2016        408.0
4  510000  3c974920a76ac9c1         1     2016        610.0


/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
# 删除province

X['adcode'] = X['adcode'].apply(lambda x: int(str(x)[:2]))
X['model'] = LabelEncoder().fit_transform(X['model'])

X_test = X.iloc[train_sales_search.shape[0]:, :]
X = X.iloc[:train_sales_search.shape[0], :]

del X_test['salesVolume']
print(X_test.head())

columns = list(X.columns)
columns.remove('salesVolume')

from sklearn.model_selection import train_test_split
X, y = X[columns], np.log1p(X['salesVolume'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=23)




   adcode  model  regMonth  regYear
0      31     13         1     2018
1      53     13         1     2018
2      15     13         1     2018
3      11     13         1     2018
4      51     13         1     2018


In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

#xgboost
def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

#lightgbm
def rmspe_lgb(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat), False

#sklearn interface
def rmspe_gscv(y, yhat):
    y = np.expm1(y)
    yhat = np.expm1(yhat)
    return rmspe(y, yhat)

rmspe_score = make_scorer(rmspe_gscv)
kfold = KFold(n_splits=5, random_state=23)



In [6]:
def xgb_model(X_train, X_valid, y_train, y_valid):
    """
    xgb 模型
    :param X_train:
    :param X_valid:
    :param y_train:
    :param y_valid:
    :param X_test_id:
    :return:
    """
    import pandas as pd
    import numpy as np
    import xgboost as xgb

    dtrain = xgb.DMatrix(X_train, label=y_train.values)
    dvalid = xgb.DMatrix(X_valid, label=y_valid.values)

    # ########################################## Tuning Paramters ##########################################
    xgb_best_params = {}
    params = {'booster': 'gbtree',
              'objective': 'reg:squarederror',
              'max_depth': 6,
              'learning_rate': 1,
              'gamma': 0,
              'min_child_weight': 1,
              'subsample': 1,
              'colsample_bytree': 1,
              'reg_alpha': 0,
              'reg_lambda ': 1,
              'random_state': 23,
              'gpu_id': 0,
              'max_bin': 16,
              'tree_method': 'gpu_exact'
              }

    # ########################################### n_estimators  ############################################
    min_merror = np.inf
    for n_estimators in range(10, 100, 10):
        params['n_estimators'] = n_estimators
        cv_results = xgb.cv(params, dtrain, nfold=3, num_boost_round=1000, early_stopping_rounds=30, feval=rmspe_xg,
                            seed=23)
        mean_error = min(cv_results['test-rmspe-mean'])

        if mean_error < min_merror:
            min_merror = mean_error
            xgb_best_params["n_estimators"] = n_estimators

    params["n_estimators"] = xgb_best_params["n_estimators"]

    # ########################################### max_depth & min_child_weight #############################
    min_merror = np.inf
    for max_depth in range(6, 11, 1):
        for min_child_weight in range(1, 6, 1):
            params['max_depth'] = max_depth
            params['min_child_weight'] = min_child_weight
            cv_results = xgb.cv(params, dtrain, nfold=3, num_boost_round=1000, early_stopping_rounds=50, feval=rmspe_xg,
                                seed=23)
            mean_error = np.argmin(cv_results['test-rmspe-mean'])

            if mean_error < min_merror:
                min_merror = mean_error
                xgb_best_params["max_depth"] = max_depth
                xgb_best_params["min_child_weight"] = min_child_weight

    params['max_depth'] = xgb_best_params['max_depth']
    params["min_child_weight"] = xgb_best_params["min_child_weight"]

    # ########################################### gamma #####################################################
    for gamma in [i / 10.0 for i in range(0, 1)]:
        params['gamma'] = gamma
        cv_results = xgb.cv(params, dtrain, nfold=3, early_stopping_rounds=50, feval=rmspe_xg, seed=23)
        mean_error = min(cv_results['test-rmspe-mean'])

        if mean_error < min_merror:
            min_merror = mean_error
            xgb_best_params["gamma"] = gamma

    params["gamma"] = xgb_best_params["gamma"]

    # ############################################# subsample & colsample_bytree ############################
    min_merror = np.inf
    for subsample in [i / 10.0 for i in range(6, 10)]:
        for colsample_bytree in [i / 10.0 for i in range(6, 10)]:
            params['subsample'] = subsample
            params['colsample_bytree'] = colsample_bytree
            cv_results = xgb.cv(params, dtrain, nfold=3, early_stopping_rounds=50, feval=rmspe_xg, seed=23)
            mean_error = min(cv_results['test-rmspe-mean'])

            if mean_error < min_merror:
                min_merror = mean_error
                xgb_best_params["subsample"] = subsample
                xgb_best_params["colsample_bytree"] = colsample_bytree

    params["subsample"] = xgb_best_params["subsample"]
    params["colsample_bytree"] = xgb_best_params["colsample_bytree"]

    # ############################################# reg_alpha ################################################
    min_merror = np.inf
    for reg_alpha in [0.8, 0.9, 1, 1.1, 1.2]:
        params['reg_alpha'] = reg_alpha
        cv_results = xgb.cv(params, dtrain, nfold=3, early_stopping_rounds=50, feval=rmspe_xg, seed=23)
        mean_error = min(cv_results['test-rmspe-mean'])

        if mean_error < min_merror:
            min_merror = mean_error
            xgb_best_params["reg_alpha"] = reg_alpha

    params["reg_alpha"] = xgb_best_params["reg_alpha"]

    # ############################################# reg_lambda ################################################
    min_merror = np.inf
    for reg_lambda in [0.8, 0.9, 1, 1.1, 1.2]:
        params['reg_lambda'] = reg_lambda
        cv_results = xgb.cv(params, dtrain, nfold=3, early_stopping_rounds=50, feval=rmspe_xg, seed=23)
        mean_error = min(cv_results['test-rmspe-mean'])

        if mean_error < min_merror:
            min_merror = mean_error
            xgb_best_params["reg_lambda"] = reg_lambda

    params["reg_lambda"] = xgb_best_params["reg_lambda"]

    # ############################################# learning_rate ################################################
    min_merror = np.inf
    for learning_rate in [0.001, 0.005, 0.01, 0.03, 0.05]:
        params['learning_rate'] = learning_rate
        cv_results = xgb.cv(params, dtrain, nfold=3, early_stopping_rounds=50, feval=rmspe_xg, seed=23)
        mean_error = min(cv_results['test-rmspe-mean'])

        if mean_error < min_merror:
            min_merror = mean_error
            xgb_best_params["learning_rate"] = learning_rate

    params["learning_rate"] = xgb_best_params["learning_rate"]

    bst_params = {"objective": params['objective'],
                  "booster": params['booster'],
                  "eta": 0.3,
                  "max_depth": params['max_depth'],
                  'min_child_weight': params['min_child_weight'],
                  "subsample": params['subsample'],
                  "colsample_bytree": params['colsample_bytree'],
                  "reg_alpha": params['reg_alpha'],
                  "silent": 1,
                  "seed": 2019,
                  "gpu_id": params['gpu_id'],
                  "max_bin": params['max_bin'],
                  "tree_method": params['tree_method']
                  }

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    xgb_model = xgb.train(bst_params, dtrain, num_boost_round=1000, evals=watchlist, early_stopping_rounds=100,
                          feval=rmspe_xg, verbose_eval=True)
    print("Validating")
    yhat = xgb_model.predict(xgb.DMatrix(X_valid))
    error = rmspe(np.expm1(y_valid.values), np.expm1(yhat))
    print('RMSPE: {:.6f}'.format(error))

xgb_model(X_train, X_valid, y_train,y_valid)

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return getattr(obj, method)(*args, **kwds)


[0]	train-rmse:3.88865	eval-rmse:3.89556	train-rmspe:0.969051	eval-rmspe:0.968478
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:2.78492	eval-rmse:2.7916	train-rmspe:0.907832	eval-rmspe:0.906479
[2]	train-rmse:2.02611	eval-rmse:2.03426	train-rmspe:0.829419	eval-rmspe:0.828267
[3]	train-rmse:1.51617	eval-rmse:1.52988	train-rmspe:0.780363	eval-rmspe:0.789346
[4]	train-rmse:1.15867	eval-rmse:1.17336	train-rmspe:0.756433	eval-rmspe:0.732812
[5]	train-rmse:0.920941	eval-rmse:0.942021	train-rmspe:0.718336	eval-rmspe:0.720333
[6]	train-rmse:0.754301	eval-rmse:0.777749	train-rmspe:0.665444	eval-rmspe:0.711323
[7]	train-rmse:0.659594	eval-rmse:0.683499	train-rmspe:0.676871	eval-rmspe:0.742981
[8]	train-rmse:0.598845	eval-rmse:0.627509	train-rmspe:0.695459	eval-rmspe:0.745061
[9]	train-rmse:0.550993	eval-rmse:0.57989	train-rmspe:0.697786	eval-rmspe:0.745578
[10]	train-rmse:0.525913	e